In [1]:
import numpy as np
import pandas as pd

In [2]:
import scipy.optimize as opt
import plotly.graph_objects as go
import chart_studio as cs
from plotly.offline import iplot

## Загружаем датасет

In [3]:
data_us = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')

In [4]:
data_us.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


### Видим, что нули только в поле fips, удалим строки, которые не вносят болшой вклад

In [5]:
data_us.isnull().sum()

date        0
county      0
state       0
fips      716
cases       0
deaths      0
dtype: int64

In [10]:
data_us[data_us['fips'].isnull()].cases.max()

103208

In [ ]:
china = table[table['Country_Region'].apply(lambda x : x == 'China' or x == 'Mainland China')]

In [17]:
data_us_without_nan = data_us.apply(lambda x, y, z, a, b, c : (y == 'New York City' and z == 'New York') )

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0
...,...,...,...,...,...,...
53843,2020-04-12,Sublette,Wyoming,56035.0,1,0
53844,2020-04-12,Sweetwater,Wyoming,56037.0,7,0
53845,2020-04-12,Teton,Wyoming,56039.0,56,0
53846,2020-04-12,Uinta,Wyoming,56041.0,4,0


In [8]:
data_us[data_us['fips'].isnull()].cases.max()

103208

In [ ]:
data_us = data_us.dropna()

In [ ]:
counties = data_us['fips'].unique()
len(counties)

## Создадим массив с данными по каждому округу

In [ ]:
data_per_county = []

for ct in counties:
    county_name = data_us[data_us['fips'] == ct]['county'].iloc[0]
    work_copy = data_us[data_us['fips'] == ct][['county', 'date', 'cases', 'deaths']]
    temp = (work_copy.groupby('date').aggregate(sum))
    temp['county'] = county_name
    if len(temp) > 5:
        data_per_county.append(temp)        

In [ ]:
len(data_per_county)

### Посмотрим на приближаемость экспонентой в трех первых округов

In [ ]:
def exp(x, A, B, C):
    return A * np.exp(B * (x - C) ** 2)

In [ ]:
figs = []

for ct in data_per_county[:3]:
    dates = ct.index #array of dates
    confirmed = ct.cases #array of confirmed
    dead = ct.deaths
    county_name = ct.county[0]
    
    # Аппроксимируем аргументы функции
    popt, pcov = opt.curve_fit(exp, np.arange(confirmed.count()), confirmed,  (1e5, -1e-2, 45), maxfev=10**6)
    A, B, C = popt
    print(county_name, ': A = ', A, ', B = ', B, ' C = ', C)
    fig = go.Figure()

    fig.add_trace(go.Bar(x=dates, y=confirmed, name="Подтвержденные случаи", marker_color='orange'))

    fig.add_trace(go.Scatter(x=dates, y=exp(np.arange(confirmed.count()), A, B, C), name='Аппроксимация', marker_color='red'))
                               
    fig.add_trace(go.Bar(x=dates, y=dead, name="Умершие", marker_color='black'))

    fig.update_layout(
        title_text="Коронавирус в " + county_name,
        title_font_size=20,
        xaxis_title="Дата",
        yaxis_title="Число людей",
        barmode='overlay'
    )
    figs.append(fig)

In [ ]:
cs.plotly.iplot(figs[0])

In [ ]:
cs.plotly.iplot(figs[1])

In [ ]:
cs.plotly.iplot(figs[2])

### Видим, что не во всех округах остался экспоненциальный рост, поэтому точность предсказания будет зависеть от количества округ, которые ушли с экспоненты

In [ ]:
arr_exps = []
i = 0
for ct in data_per_county:
    dates = ct.index #array of dates
    confirmed = ct.cases #array of confirmed
    county_name = ct.county[0]
    index = np.arange(confirmed.count())
    
    # Аппроксимируем аргументы функции
    popt, pcov = opt.curve_fit(exp, index, confirmed,  (1e5, -1e-2, 45), maxfev=10**6)
    A, B, C = popt
    
    # Выводим аргументы для функции роста числа зараженных в данном округе
    print(i, county_name, ': A = ', A, ', B = ', B, ' C = ', C)
    
    # Так как наша аппроксимирую
    arr_exps.append(max(exp(index[-1] + 4, A, B, C), confirmed[-1]))
    print('Состояние на 12.04: ', confirmed[-1])
    print('Предсказание на 16.04: ', arr_exps[-1], '\n')
    i += 1

In [ ]:
exps = np.array(arr_exps)
round(exps.sum())

## Итог:

Всего на 16.04 имеем аппроксимацию в 626354 зараженных